In [2]:
print("OK!")

OK!


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader,PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\Samiran A\anaconda3\envs\mchatbot\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
PINECONE_API_KEY = ""
PINECONE_API_ENV = ""


In [4]:
# Extrach data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [5]:
import pypdf

In [6]:
extracted_data = load_pdf("data/")

In [7]:
# extracted_data

In [8]:
# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500,chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("length of my chunks:",len(text_chunks))

length of my chunks: 7020


In [10]:
# Download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [23]:
query_result

[-0.034477222710847855,
 0.031023185700178146,
 0.006734990049153566,
 0.026108937337994576,
 -0.03936203941702843,
 -0.16030247509479523,
 0.06692399084568024,
 -0.0064415158703923225,
 -0.04745062440633774,
 0.014758840203285217,
 0.07087533175945282,
 0.05552753433585167,
 0.01919342391192913,
 -0.026251353323459625,
 -0.010109438560903072,
 -0.02694055624306202,
 0.02230747416615486,
 -0.022226637229323387,
 -0.14969265460968018,
 -0.01749308779835701,
 0.007676266133785248,
 0.05435233190655708,
 0.003254504408687353,
 0.03172597661614418,
 -0.0846213847398758,
 -0.0294059868901968,
 0.051595672965049744,
 0.0481240376830101,
 -0.0033147623762488365,
 -0.05827918276190758,
 0.041969288140535355,
 0.02221066877245903,
 0.12818889319896698,
 -0.022338978946208954,
 -0.011656207032501698,
 0.06292837858200073,
 -0.032876212149858475,
 -0.09122608602046967,
 -0.031175455078482628,
 0.05269963666796684,
 0.04703482985496521,
 -0.08420305699110031,
 -0.03005620278418064,
 -0.02074477262

In [31]:
# # Initializing the Pinecone
# pinecone.init(api_key=PINECONE_API_KEY,environment=PINECONE_API_ENV)


# index_name = "medical-chatbot"

# # Creating Embeddings for each of the text chunks & storing in Pinecone Vector DB
# docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [18]:
# Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,environment=PINECONE_API_ENV)

index_name = "medical-chatbot"

In [19]:
# Assuming we already have an index we can load it like this
docsearch = Pinecone.from_existing_index(index_name=index_name,embedding=embeddings)

query = "What are Allergies"

docs = docsearch.similarity_search(query=query, k = 3)

print("Result",docs) # This gives only top 3 ranked result. To get the correct answer we call out LLM model

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE", metadata={}), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drug

In [20]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helful answer:
"""

In [21]:
PROMPT = PromptTemplate(template=prompt_template,input_variables = ["context","question"])
chain_type_kwargs = {"prompt" : PROMPT}

In [22]:
## LOADING THE MODEL
llm = CTransformers(model = "model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type = "llama",
                    config = {'max_new_tokens' : 512,
                              'temperature' : 0.8})

In [23]:
## Creating Q/A object
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = docsearch.as_retriever(serach_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [25]:
while True:
    user_input=input(f"Input Prompt")
    result=qa({"query":user_input})
    print("Response : ", result["result"])

Response :  

Acneous material.


Acne material. Acnevermatter.
I don'Acneeds.


Acne, but if you can be filled with benign content.

Acneous material.


I don'I apologize and tissue that acnes.
I don'Acne.
Acneuids.
Acneerial contents.




acid content and/rincip> Acneeds.
I don’s.

The maincontent to follow up of an organic matter.
Acne,  Of course of its a substance. However, as of pusually cyst content will be closed airboring.







Acne contents.
Acne material. Acneat
Sorry 
I don'Acneous material.






I don'I don’s.







Acneous material. Acne.


We hope this page  I don'Acne.


















Sorry matter. The information
The type of contents. 
I don’Acidosis clogging, you may not applicable to the cyst.
Acne contents.
Acneerial material.
Acneeds.





I'I don'Acneous material.

I do you have a sacs.
Acneeds.
Acneuide.



acids. I don’s.
I don'The information.



Acneeds.



Acneerial content:content goes on top |The maincontent is not applicable to the answer: content;
A

KeyboardInterrupt: 